In [ ]:
!apt-get install -y libimage-exiftool-perl

In [ ]:
import cv2
import math
import os
import numpy as np
import subprocess
from google.colab.patches import cv2_imshow

# Downsampling the visible image (k%)
def downsample_image(image, downsample_percentage):
    height, width = image.shape[:2]
    new_width = int(width * downsample_percentage / 100)
    new_height = int(height * downsample_percentage / 100)
    downsampled_V_image = cv2.resize(image, (new_width, new_height))
    return downsampled_V_image

# Cropping the center of the downsampled V image
def center_crop(image, template_shape):
    template_height, template_width = template_shape
    height, width = image.shape[:2]
    start_y = max(0, (height - template_height) // 2)
    start_x = max(0, (width - template_width) // 2)
    end_y = min(height, start_y + template_height)
    end_x = min(width, start_x + template_width)
    cropped_image = image[start_y:end_y, start_x:end_x]
    return cropped_image

# Pincushion distortion correction for T and V
def correct_image(source_image, strength, zoom, output_file):
    image_height, image_width, _ = source_image.shape
    half_width = image_width / 2
    half_height = image_height / 2
    if strength == 0:
        strength = 0.00001
    correction_radius = math.sqrt(image_width ** 2 + image_height ** 2) / strength
    corrected_image = source_image.copy()
    for y in range(image_height):
        for x in range(image_width):
            new_x = x - half_width
            new_y = y - half_height
            distance = math.sqrt(new_x ** 2 + new_y ** 2)
            r = distance / correction_radius
            if r == 0:
                theta = 1
            else:
                theta = math.atan(r) / r
            source_x = half_width + theta * new_x * zoom
            source_y = half_height + theta * new_y * zoom
            source_x = int(source_x)
            source_y = int(source_y)
            if source_x >= 0 and source_x < image_width and source_y >= 0 and source_y < image_height:
                source_pixel = source_image[source_y, source_x]
            else:
                source_pixel = [0, 0, 0]  # Default color for out-of-bounds pixels
            corrected_image[y, x] = source_pixel
    cv2.imwrite(output_file, corrected_image)
    return corrected_image

# ECC to align images
def align_images(visible_img, infrared_img, output_path):
    try:
        # Convert images to grayscale
        im1_gray = cv2.cvtColor(infrared_img, cv2.COLOR_BGR2GRAY)
        im2_gray = cv2.cvtColor(visible_img, cv2.COLOR_BGR2GRAY)

        # Find size of image1
        sz = infrared_img.shape

        # Define the motion model
        warp_mode = cv2.MOTION_EUCLIDEAN

        # Define 2x3 or 3x3 matrices and initialize the matrix to identity
        if warp_mode == cv2.MOTION_HOMOGRAPHY:
            warp_matrix = np.eye(3, 3, dtype=np.float32)
        else:
            warp_matrix = np.eye(2, 3, dtype=np.float32)

        # Specify the number of iterations
        number_of_iterations = 5000

        # Specify the threshold of the increment
        # in the correlation coefficient between two iterations
        termination_eps = 1e-10

        # Define termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

        # Run the ECC algorithm. The results are stored in warp_matrix.
        (cc, warp_matrix) = cv2.findTransformECC(im1_gray, im2_gray, warp_matrix, warp_mode, criteria)

        if warp_mode == cv2.MOTION_HOMOGRAPHY:
            # Use warpPerspective for Homography
            im2_aligned = cv2.warpPerspective(visible_img, warp_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else:
            # Use warpAffine for Translation, Euclidean and Affine
            im2_aligned = cv2.warpAffine(visible_img, warp_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

        # Save the aligned image
        cv2.imwrite(output_path, im2_aligned)

    except Exception as e:
        print(f"Error processing images: {e}")



# Load the input images
folder = '/content/drive/MyDrive/ColabNotebooks/data-fusion2/RGB/'
for i in range(313,324):
    j = 2 * i + 1
    k = i + 1
    visible_path = os.path.join(folder, f'sf_1 ({k}).jpg')
    if j < 10:
        thermal_path = f'/content/drive/MyDrive/ColabNotebooks/data-fusion2/T/DJI_000{j}_R.JPG'
    elif j >= 10 and j < 100:
        thermal_path = f'/content/drive/MyDrive/ColabNotebooks/data-fusion2/T/DJI_00{j}_R.JPG'
    else:
        thermal_path = f'/content/drive/MyDrive/ColabNotebooks/data-fusion2/T/DJI_0{j}_R.JPG'

    visible_img = cv2.imread(visible_path)
    thermal_img = cv2.imread(thermal_path)
    if visible_img is None or thermal_img is None:
        raise ValueError("One or both of the input images could not be loaded.")

    # Outputs
    visible_output = f'/content/drive/MyDrive/ColabNotebooks/data-fusion2/aligned-v/DJI_{k}_V.JPG'
    thermal_output = f'/content/drive/MyDrive/ColabNotebooks/data-fusion2/aligned-t/DJI_{k}_T.JPG'
    piw_corrected_V_output = f'/content/drive/MyDrive/Colab/Data_colab/piw_corrected_V/piw_DJI_{k}_V.JPG'

    # Downsampling the visible image (k%)
    downsampled_V_image = downsample_image(visible_img, downsample_percentage = 28)

    # Cropping the center of the downsampled V image
    thermal_image_shape = thermal_img.shape[:2]
    V_cropped_pair = center_crop(downsampled_V_image, thermal_image_shape)

    # Parameters for Pincushion correction
    visible_strength = 1.2
    visible_zoom = 1.0
    thermal_strength = 0.00001
    thermal_zoom = 0.96
    # Pincushion correction for V
    corrected_visible_image = correct_image(V_cropped_pair, visible_strength, visible_zoom, visible_output)
    # Pincushion correction for T
    corrected_thermal_image = correct_image(thermal_img, thermal_strength, thermal_zoom, thermal_output)

    # ECC to align V images
    #align_images(corrected_visible_image, corrected_thermal_image, visible_output)

    # Copy Exif data from the original images to the output images
    subprocess.run(['exiftool', '-tagsFromFile', thermal_path, thermal_output])
    subprocess.run(['exiftool', '-tagsFromFile', visible_path, visible_output])
    print(f"Image is processed.")
